In [8]:
## Import necessary libraries
import os
import pandas as pd
import numpy as np
import math

In [9]:
# Duplicate Handling Tool
def check_dupl(in_df, var_list=['id', 'time']):
    # Identify duplicate observations; keep=False marks all duplicates as True
    
    duplicates = in_df.duplicated(subset=var_list, keep=False)
    
    # Use boolean indexing to list all duplicate observations
    in_df_duplicates = in_df[duplicates]
    
    # Display duplicate observations
    print(in_df_duplicates)

# Creating Column Headers
Using the CLSPRC file as a reference to create the column headers.

In [10]:
#文件读取
head_path = "./CLSPRC_yr.xlsx"

head_df = pd.read_excel(head_path, engine='openpyxl')

# head_df

/Users/charles/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
head_df = head_df.drop([0,1]) #drop掉两个备注行

# head_df

In [12]:
stkcd_values = head_df['Stkcd'].unique()

stkcd_df =  pd.DataFrame(stkcd_values, columns=['id'])

stkcd_df['id'] = stkcd_df['id'].astype('int')

#创建一个key列进行匹配
stkcd_df['key'] = 1


# stkcd_df

In [13]:
#创建一个从2000-2021年的年份列
year_end_values = pd.date_range(start='2000-01-01', end='2022-01-01', freq='Y')

year_start_values = pd.date_range(start='2000-01-01', end='2021-01-01', freq='YS')

year_df = pd.DataFrame({'year_start': year_start_values, 'year_end': year_end_values})

year_df['time'] = year_df['year_start'].dt.year #生成一列年份列
year_df['key'] = 1

# year_df

In [14]:
# 将股票列和时间列合并在一起
merged_df = pd.merge(stkcd_df, year_df, on='key').drop('key', axis=1)

# merged_df


In [15]:
# 表头文件单独导出
merged_df.to_csv('./merged/head_yr.csv', index=False)

In [16]:
# 替换raw_df
raw_df = merged_df

In [17]:
raw_df

,id,year_start,year_end,time
0,1,2000-01-01,2000-12-31,2000
1,1,2001-01-01,2001-12-31,2001
2,1,2002-01-01,2002-12-31,2002
3,1,2003-01-01,2003-12-31,2003
4,1,2004-01-01,2004-12-31,2004
...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017
125154,900957,2018-01-01,2018-12-31,2018
125155,900957,2019-01-01,2019-12-31,2019
125156,900957,2020-01-01,2020-12-31,2020


# Matching Typhoon Data

In [18]:
# 读取台风数据
typh_path = './TYPH.xlsx'

typh_df = pd.read_excel(typh_path, engine='openpyxl')

# typh_df

In [19]:
typh_df = typh_df[['code', 'year', 'typh', 'obj_id', 'typh_date', 'typh_id', 'typh_name', 'typh_strg', 'typh_spd', 'typh_longt', 'typh_lat', 'typh_no']]

#重命名
typh_df = typh_df.rename(columns={'code': 'id', 'year': 'time'})

# typh_df

In [20]:
# check_dupl(typh_df) # 检查发现有重复值

# 删除重复项，只保留首次出现的观测值
typh_df = typh_df.drop_duplicates(subset=['id', 'time'], keep='first')

In [21]:
# 将台风数据和表头合并在一起

merged_df = pd.merge(raw_df, typh_df, on=['id', 'time'], how='left')

#进行导出检查
# merged_df.to_csv('./merged/typh_merged.csv', index=False)

# merged_df

In [22]:
# 将台风的缺失值改为0
merged_df['typh'].fillna(0, inplace=True)
# merged_df

In [23]:
# 替换raw_df

raw_df = merged_df

raw_df

,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,typh_spd,typh_longt,typh_lat,typh_no
0,1,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2003-01-01,2003-12-31,2003,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125154,900957,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125155,900957,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125156,900957,2020-01-01,2020-12-31,2020,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Matching Latitude Data

In [121]:
lat_path = './LATITUDE.xlsx'

lat_df = pd.read_excel(lat_path, engine='openpyxl')

# lat_df

,Symbol,ShortName,EndDate,ListedCoID,SecurityID,OfficeAddress,Lng,Lat
0,股票代码,股票简称,统计截止日期,上市公司ID,证券ID,公司办公地址,办公地经度,办公地纬度
1,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位
2,000001,平安银行,2022-12-31,101704,201000000001,广东省深圳市深南东路5047号;广东省深圳市福田区益田路5023号平安金融中心B座,114.107740,22.540680
3,000002,万科A,2022-12-31,101775,201000000002,广东省深圳市盐田区大梅沙环梅路33号万科中心,114.302979,22.596869
4,000004,ST 国华,2022-12-31,102294,201000000004,广东省深圳市福田区梅林街道孖岭社区凯丰路10号翠林大厦12层,114.063957,22.569613
...,...,...,...,...,...,...,...,...
5164,873223,荣亿精密,2022-12-31,10455505,201004213645,浙江省嘉兴市海盐县望海街道盐嘉公路新兴段336号,120.933342,30.590651
5165,873305,九菱科技,2022-12-31,10609393,201004232344,湖北省荆州市沙市区关沮工业园西湖路129号,112.271320,30.347850
5166,873339,恒太照明,2022-12-31,10604398,201004227383,江苏省南通市经济技术开发区复兴东路18号,120.988925,31.927625
5167,873527,夜光明,2022-12-31,10197587,201004226643,浙江省台州市经济开发区滨海工业园海丰路2355号,121.522150,28.603140


In [122]:
lat_df = lat_df.drop([0,1])

In [130]:
lat_df['id'] = lat_df['Symbol'].astype('int')

lat_df['Lat'] = lat_df['Lat'].astype('float')

lat_df = lat_df.rename(columns={'Lat':"lat"})

lat_df = lat_df.drop_duplicates(subset=['id'], keep='first')

lat_df = lat_df[['id', 'lat']]

lat_df

,id,lat
2,1,22.540680
3,2,22.596869
4,4,22.569613
5,5,22.544228
6,6,22.535330
...,...,...
5164,873223,30.590651
5165,873305,30.347850
5166,873339,31.927625
5167,873527,28.603140


In [131]:
merged_df = pd.merge(raw_df, lat_df, on=['id'], how='left')

# merged_df

,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,...,industry,industry_id,soe,gamma,delta,eps_1,eps_2,cef,ln_size,lat
0,1,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,...,金融、保险业,I,0.0,0.342502,0.559031,0.260328,0.260328,0.014774,24.931348,22.54068
1,1,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,...,金融、保险业,I,0.0,0.342502,0.800832,0.206782,0.206782,0.014181,25.511815,22.54068
2,1,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,...,金融、保险业,I,0.0,0.342502,0.466894,0.222130,0.222130,0.017877,25.836255,22.54068
3,1,2003-01-01,2003-12-31,2003,0.0,NaN,NaN,NaN,NaN,NaN,...,金融、保险业,I,0.0,0.342502,0.466894,0.162680,0.162680,0.015320,25.985184,22.54068
4,1,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,...,金融、保险业,I,0.0,0.342502,0.225093,0.244750,0.230000,0.025393,26.042789,22.54068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.017639,0.000000,0.086005,0.086005,0.071100,NaN,NaN
125154,900957,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.017639,0.000000,0.079327,0.079327,0.075987,NaN,NaN
125155,900957,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.017639,0.000000,0.072611,0.072611,0.106360,NaN,NaN
125156,900957,2020-01-01,2020-12-31,2020,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.017639,0.000000,0.062632,0.062632,0.102047,NaN,NaN


In [132]:
raw_df = merged_df

# Matching RND Data

In [24]:
# 导入RND数据
rnd_path = './RND_yr.xlsx'

rnd_df = pd.read_excel(rnd_path, engine='openpyxl')

# rnd_df

In [25]:
# drop掉备注列
rnd_df = rnd_df.drop([0,1])

rnd_df

,Symbol,EndDate,Source,StateTypeCode,RDSpendSumRatio
2,000002,2018-12-31,0,1,NaN
3,000002,2019-12-31,0,1,NaN
4,000002,2020-12-31,0,1,NaN
5,000002,2021-12-31,0,1,NaN
6,000004,2010-12-31,0,1,NaN
...,...,...,...,...,...
50776,873527,2020-12-31,0,1,3.54
50777,873527,2021-12-31,0,1,3.42
50778,873527,2022-12-31,0,1,4.16
50779,873593,2022-12-31,0,1,4.8


In [26]:
# 只留下StateTypeCode为1的行

rnd_df = rnd_df[rnd_df['StateTypeCode'] == 1]

In [27]:
# 去掉非年终的行
rnd_df['EndDate'] = pd.to_datetime(rnd_df['EndDate'])
rnd_df = rnd_df[rnd_df['EndDate'].dt.month.eq(12) & rnd_df['EndDate'].dt.day.eq(31)]

/var/folders/20/dhknrks5505f4hwlc2p9vg500000gn/T/ipykernel_7789/2339810936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rnd_df['EndDate'] = pd.to_datetime(rnd_df['EndDate'])


In [28]:
# 转化符号
rnd_df['id'] = rnd_df['Symbol'].astype('int')

# 转化时间
rnd_df['time'] = pd.to_datetime(rnd_df['EndDate']).dt.year

# 重命名
rnd_df = rnd_df.rename(columns={'RDSpendSumRatio':'rnd'})

# 只留下想要的列
rnd_df = rnd_df[['id', 'time', 'rnd']]

# rnd_df

In [29]:
check_dupl(rnd_df)

Empty DataFrame
Columns: [id, time, rnd]
Index: []


In [30]:
# merge RND和表头
merged_df = pd.merge(raw_df, rnd_df, on=['id', 'time'], how='left')

merged_df.to_csv('./merged/rnd_merged.csv', index=False)

merged_df

,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,typh_spd,typh_longt,typh_lat,typh_no,rnd
0,1,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2003-01-01,2003-12-31,2003,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125154,900957,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125155,900957,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125156,900957,2020-01-01,2020-12-31,2020,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# 替换raw_df
raw_df = merged_df

# Matching BETA Data

In [32]:
# 读取beta数据
beta_path = "./BETA_yr.xlsx"

beta_df = pd.read_excel(beta_path, engine="openpyxl")

# beta_df

/Users/charles/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [33]:
beta_df = beta_df.drop([0,1])

beta_df

,Stkcd,Trddt,IsNotBSE,Betavals
2,000001,1999,1.0,1.10424
3,000001,2000,1.0,0.78727
4,000001,2001,1.0,0.9387
5,000001,2002,1.0,1.00167
6,000001,2003,1.0,1.34287
...,...,...,...,...
81248,873679,2023,0.0,-3.19084
81249,873693,2023,0.0,55.80276
81250,873703,2023,0.0,-12.02707
81251,873726,2023,0.0,0.64078


In [34]:
# 剔除北交所股票为1
beta_df = beta_df[beta_df['IsNotBSE'].eq(1)]

In [35]:
beta_df['id'] = beta_df['Stkcd'].astype("int")

beta_df['time'] = pd.to_datetime(beta_df['Trddt']).dt.year

beta_df = beta_df.rename(columns={'Betavals': 'beta'})

beta_df = beta_df[['id', 'time', 'beta']]

# beta_df

/var/folders/20/dhknrks5505f4hwlc2p9vg500000gn/T/ipykernel_7789/836211291.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beta_df['id'] = beta_df['Stkcd'].astype("int")
/var/folders/20/dhknrks5505f4hwlc2p9vg500000gn/T/ipykernel_7789/836211291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beta_df['time'] = pd.to_datetime(beta_df['Trddt']).dt.year


In [36]:
check_dupl(beta_df)

Empty DataFrame
Columns: [id, time, beta]
Index: []


In [37]:
merged_df = pd.merge(raw_df, beta_df, on=['id', 'time'], how="left")

merged_df.to_csv('./merged/beta_merged.csv', index=False)

# merged_df

In [38]:
# 替换raw_df
raw_df = merged_df

raw_df

,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,typh_spd,typh_longt,typh_lat,typh_no,rnd,beta
0,1,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78727
1,1,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9387
2,1,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00167
3,1,2003-01-01,2003-12-31,2003,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.34287
4,1,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.87204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125154,900957,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125155,900957,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125156,900957,2020-01-01,2020-12-31,2020,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# # Matching CLSPRC (Closing Price) Data

In [39]:
prc_path = "./CLSPRC_yr.xlsx"

prc_df = pd.read_excel(prc_path, engine="openpyxl")

# prc_df

/Users/charles/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [40]:
prc_df = prc_df.drop([0,1])

# prc_df

In [41]:
# 推后一年
prc_df['date_plus1yr'] = pd.to_datetime(prc_df['Trdynt']) + pd.DateOffset(years=1)

# prc_df

In [42]:
prc_df['id'] = prc_df['Stkcd'].astype('int')

prc_df['time'] = prc_df['date_plus1yr'].dt.year

prc_df = prc_df.rename(columns={'Yclsprc' : 'clsprc'})

prc_df = prc_df[['id', 'time', 'clsprc']]

# prc_df

In [43]:
# 检查重复项
check_dupl(prc_df)

Empty DataFrame
Columns: [id, time, clsprc]
Index: []


In [44]:
merged_df = pd.merge(raw_df, prc_df, on = ['id', 'time'], how='left')

merged_df.to_csv('./merged/prc_merged.csv', index=False)

# merged_df

In [45]:
raw_df = merged_df

# # Matching DPS(Dividend per share), LEV (Leverage) Data

In [46]:
dps_path = './DPS-LEV_yr.xlsx'

dps_df = pd.read_excel(dps_path, engine='openpyxl')

# dps_df

/Users/charles/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [47]:
dps_df = dps_df.drop([0,1])

# dps_df

In [48]:
dps_df['id'] = dps_df['Stkcd'].astype('int')

dps_df['time'] = pd.to_datetime(dps_df['Accper']).dt.year

dps_df = dps_df.rename(columns={'T20497': 'dps', 'T30100': 'lev'})

dps_df = dps_df[['id', 'time','dps', 'lev']]

# dps_df

In [49]:
check_dupl(dps_df)

Empty DataFrame
Columns: [id, time, dps, lev]
Index: []


In [50]:
merged_df = pd.merge(raw_df, dps_df, on = ['id', 'time'], how='left')

# merged_df

In [51]:
raw_df = merged_df

# Matching EPS(Earnings per share), PM(PM ratio) Data

In [52]:
eps_path = './EPS-BM_yr.xlsx'

eps_df = pd.read_excel(eps_path, engine='openpyxl')

# eps_df

/Users/charles/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [53]:
eps_df = eps_df.drop([0,1])

# eps_df

In [54]:
eps_df['id'] = eps_df['Stkcd'].astype('int')

eps_df['time'] = pd.to_datetime(eps_df['Accper']).dt.year

eps_df = eps_df.rename(columns={'T60200': 'eps', 'Bktomk': 'bm'})

eps_df = eps_df[['id', 'time', 'eps', 'bm']]

# eps_df

In [55]:
check_dupl(eps_df)

Empty DataFrame
Columns: [id, time, eps, bm]
Index: []


In [56]:
merged_df = pd.merge(raw_df, eps_df, on=['id', 'time'], how='left')

merged_df.to_csv('./merged/eps_merged.csv', index=False)

# merged_df

In [57]:
raw_df = merged_df

# Matching ROA, Asset Data

In [58]:
roa_path = "./ROA-ASSET_yr.xlsx"

roa_df = pd.read_excel(roa_path, engine="openpyxl")

# roa_df

In [59]:
roa_df = roa_df.drop([0,1])

# roa_df

In [60]:
roa_df['id'] = roa_df['Stkcd'].astype('int')

roa_df['time'] = pd.to_datetime(roa_df['Accper']).dt.year

roa_df = roa_df.rename(columns={'A100000': 'asset', 'A200000': 'debt', 'A300000': 'equity', 'T40401': 'roa'})

roa_df = roa_df[['id', 'time','asset', 'debt', 'equity', 'roa']]

roa_df

,id,time,asset,debt,equity,roa
2,1,1998,39399858617.260002,35723495376.989998,3676648292.92,0.019397
3,1,1999,45868972050,42968141344,2900830706,0.012104
4,1,2000,67227499769,62488616114,4738883655,0.007535
5,1,2001,120126983351,116499314559,3627668792,0.003349
6,1,2002,166166379400,162398358027,3768021373,0.002601
...,...,...,...,...,...,...
59709,873703,2022,832715165.18,313980276.4,518734888.78,0.157629
59710,873726,2022,531876839.77,253799136.97,278077702.8,0.166311
59711,873806,2021,5124748447.04,4135552377.66,906649287.87,0.023831
59712,873806,2022,3780016983.88,2829279397.09,946226273.67,0.027077


In [61]:
check_dupl(roa_df)

Empty DataFrame
Columns: [id, time, asset, debt, equity, roa]
Index: []


In [62]:
merged_df = pd.merge(raw_df, roa_df, on=['id', 'time'], how='left')

merged_df.to_csv('./merged/roa_merged.csv', index = False)

# merged_df

In [63]:
raw_df = merged_df

# Matching Current Ratio Data

In [64]:
cash_path = './CASH_yr.xlsx'

cash_df = pd.read_excel(cash_path, engine='openpyxl')

# cash_df

/Users/charles/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [65]:
cash_df = cash_df.drop([0,1])


# cash_df

In [66]:
cash_df = cash_df[cash_df['StateType'] == "A"]

cash_df['time'] = pd.to_datetime(cash_df['EndDate']).dt.year
cash_df['id'] = cash_df['Symbol'].astype('int')

cash_df = cash_df.rename(columns={"TangibleAssetRatio": "tang_asset_ratio", "CurrentRatio": "current_ratio", "ProfitsVolatility": "profit_volability"})

cash_df = cash_df[['id', 'time', 'tang_asset_ratio', 'current_ratio', 'profit_volability']]

# cash_df

In [67]:
merged_df = pd.merge(raw_df, cash_df, on=['id', 'time'], how='left')

# merged_df

In [68]:
raw_df = merged_df

# raw_df

# Matching cash ratio data

Quarterly data, taking year-end data as the value

In [69]:
cashq_path = './CASH_qt.xlsx'

cashq_df = pd.read_excel(cashq_path, engine='openpyxl')

# cashq_df

In [70]:
cashq_df = cashq_df.drop([0,1])

# cashq_df

In [71]:
cashq_df['Accper'] = pd.to_datetime(cashq_df['Accper'])

cashq_df = cashq_df[cashq_df['Typrep'] == "A"]
cashq_df = cashq_df[cashq_df['Accper'].dt.month.eq(12) & cashq_df['Accper'].dt.day.eq(31)]

cashq_df['id'] = cashq_df['Stkcd'].astype('int')
cashq_df['time'] = cashq_df['Accper'].dt.year

cashq_df = cashq_df.rename(columns={'F010401A': 'cash_ratio'})

# cashq_df

In [72]:
cashq_df = cashq_df[['id', 'time', 'cash_ratio']]

# cashq_df

In [73]:
merged_df = pd.merge(raw_df, cashq_df, on=['id', 'time'], how='left')

# merged_df

In [74]:
raw_df = merged_df

# Industry and SOE type treatment

In [75]:
# 读取数据
soe_path = './SOE-IND_yr.xlsx'

soe_df = pd.read_excel(soe_path, engine='openpyxl')

soe_df

,证券代码,证券名称,证监会行业,证监会行业代码,公司性质（同花顺）
0,430017.BJ,星昊医药,制造业,C,民营企业
1,430047.BJ,诺思兰德,制造业,C,民营企业
2,430090.BJ,同辉信息,制造业,C,民营企业
3,430139.BJ,华岭股份,制造业,C,其他
4,430198.BJ,微创光电,信息技术业,G,省属国资控股
...,...,...,...,...,...
5350,301578.SZ,辰奕智能,--,--,民营企业
5351,301589.SZ,诺瓦星云,--,--,民营企业
5352,301591.SZ,肯特股份,--,--,民营企业
5353,NaN,NaN,NaN,NaN,NaN


In [76]:
soe_df = soe_df.drop([5354, 5353]) #剔除最后一行

soe_df

,证券代码,证券名称,证监会行业,证监会行业代码,公司性质（同花顺）
0,430017.BJ,星昊医药,制造业,C,民营企业
1,430047.BJ,诺思兰德,制造业,C,民营企业
2,430090.BJ,同辉信息,制造业,C,民营企业
3,430139.BJ,华岭股份,制造业,C,其他
4,430198.BJ,微创光电,信息技术业,G,省属国资控股
...,...,...,...,...,...
5348,301568.SZ,思泰克,--,--,民营企业
5349,301577.SZ,美信科技,--,--,民营企业
5350,301578.SZ,辰奕智能,--,--,民营企业
5351,301589.SZ,诺瓦星云,--,--,民营企业


In [77]:
# 对列进行重命名
soe_df = soe_df.rename(columns={'证券代码': 'id', '证监会行业': 'industry', '证监会行业代码': 'industry_id', '公司性质（同花顺）': 'property'})

# 处理ID
soe_df['id'] = soe_df['id'].str[:6].astype('int')

soe_df

,id,证券名称,industry,industry_id,property
0,430017,星昊医药,制造业,C,民营企业
1,430047,诺思兰德,制造业,C,民营企业
2,430090,同辉信息,制造业,C,民营企业
3,430139,华岭股份,制造业,C,其他
4,430198,微创光电,信息技术业,G,省属国资控股
...,...,...,...,...,...
5348,301568,思泰克,--,--,民营企业
5349,301577,美信科技,--,--,民营企业
5350,301578,辰奕智能,--,--,民营企业
5351,301589,诺瓦星云,--,--,民营企业


In [78]:
# 处理公司性质数据
#查看所有可能的公司性质
print(soe_df['property'].value_counts())

# 生成是否为国企的变量
soe_df['soe'] = soe_df['property'].apply(lambda x: 1 if '国' in x else 0)

soe_df

民营企业      3534
省属国资控股     514
央企国资控股     477
地市国资控股     466
外资企业       246
其他          73
集体企业        26
其他国有        17
Name: property, dtype: int64


,id,证券名称,industry,industry_id,property,soe
0,430017,星昊医药,制造业,C,民营企业,0
1,430047,诺思兰德,制造业,C,民营企业,0
2,430090,同辉信息,制造业,C,民营企业,0
3,430139,华岭股份,制造业,C,其他,0
4,430198,微创光电,信息技术业,G,省属国资控股,1
...,...,...,...,...,...,...
5348,301568,思泰克,--,--,民营企业,0
5349,301577,美信科技,--,--,民营企业,0
5350,301578,辰奕智能,--,--,民营企业,0
5351,301589,诺瓦星云,--,--,民营企业,0


In [79]:
soe_df = soe_df[['id', 'industry', 'industry_id', 'soe']]

soe_df

,id,industry,industry_id,soe
0,430017,制造业,C,0
1,430047,制造业,C,0
2,430090,制造业,C,0
3,430139,制造业,C,0
4,430198,信息技术业,G,1
...,...,...,...,...
5348,301568,--,--,0
5349,301577,--,--,0
5350,301578,--,--,0
5351,301589,--,--,0


In [80]:
# 查看所有可能行业
print(soe_df['industry'].value_counts())

print(soe_df.groupby('industry')['industry_id'].unique())

制造业               2381
--                1556
信息技术业              375
社会服务业              180
批发和零售贸易            151
房地产业               103
交通运输、仓储业            95
建筑业                 95
电力、煤气及水的生产和供应业      91
采掘业                 82
金融、保险业              73
传播与文化产业             66
农、林、牧、渔业            56
综合类                 49
Name: industry, dtype: int64
industry
--                [--]
交通运输、仓储业           [F]
传播与文化产业            [L]
信息技术业              [G]
农、林、牧、渔业           [A]
制造业                [C]
建筑业                [E]
房地产业               [J]
批发和零售贸易            [H]
电力、煤气及水的生产和供应业     [D]
社会服务业              [K]
综合类                [M]
采掘业                [B]
金融、保险业             [I]
Name: industry_id, dtype: object


In [81]:
check_dupl(soe_df, var_list=['id'])

Empty DataFrame
Columns: [id, industry, industry_id, soe]
Index: []


In [82]:
merged_df = pd.merge(raw_df, soe_df, on=['id'], how='left')

# merged_df.to_csv('./merged/soe_merged.csv', index=False)

# merged_df

In [83]:
# 替换
raw_df = merged_df

# Matching ESG rating data

In [142]:
# 读取数据
esg_path = './ESG.xlsx'

esg_df = pd.read_excel(esg_path, engine = 'openpyxl')

# esg_df

In [143]:
esg_df = esg_df.drop([5353, 5354])

# esg_df

In [147]:
# 处理ID
esg_df['id'] = esg_df['证券代码'].str[:6].astype('int')

# 重命名
esg_df = esg_df.rename(columns={'华证ESG评级': 'esg'})

esg_df = esg_df[['id', 'esg']]

# esg_df

,id,esg
0,430017,B
1,430047,B
2,430090,CCC
3,430139,B
4,430198,B
...,...,...
5348,301568,CCC
5349,301577,CCC
5350,301578,B
5351,301589,--


In [149]:
print(esg_df['esg'].value_counts())

B      2566
BB     1490
CCC     714
BBB     437
CC       77
A        61
--        8
Name: esg, dtype: int64


In [148]:
merged_df = pd.merge(raw_df, esg_df, on=['id'], how='left')

# merged_df

,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,...,industry_id,soe,gamma,delta,eps_1,eps_2,cef,ln_size,lat,esg
0,1,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,...,I,0.0,0.342502,0.559031,0.260328,0.260328,0.014774,24.931348,22.54068,A
1,1,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,...,I,0.0,0.342502,0.800832,0.206782,0.206782,0.014181,25.511815,22.54068,A
2,1,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,...,I,0.0,0.342502,0.466894,0.222130,0.222130,0.017877,25.836255,22.54068,A
3,1,2003-01-01,2003-12-31,2003,0.0,NaN,NaN,NaN,NaN,NaN,...,I,0.0,0.342502,0.466894,0.162680,0.162680,0.015320,25.985184,22.54068,A
4,1,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,...,I,0.0,0.342502,0.225093,0.244750,0.230000,0.025393,26.042789,22.54068,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.017639,0.000000,0.086005,0.086005,0.071100,NaN,NaN,NaN
125154,900957,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.017639,0.000000,0.079327,0.079327,0.075987,NaN,NaN,NaN
125155,900957,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.017639,0.000000,0.072611,0.072611,0.106360,NaN,NaN,NaN
125156,900957,2020-01-01,2020-12-31,2020,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.017639,0.000000,0.062632,0.062632,0.102047,NaN,NaN,NaN


In [ ]:
raw_df = merged_df

# Organize data types

In [84]:
print(raw_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125158 entries, 0 to 125157
Data columns (total 32 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 125158 non-null  int64         
 1   year_start         125158 non-null  datetime64[ns]
 2   year_end           125158 non-null  datetime64[ns]
 3   time               125158 non-null  int64         
 4   typh               125158 non-null  float64       
 5   obj_id             2480 non-null    float64       
 6   typh_date          2480 non-null    float64       
 7   typh_id            2480 non-null    object        
 8   typh_name          2480 non-null    object        
 9   typh_strg          2480 non-null    float64       
 10  typh_spd           2480 non-null    float64       
 11  typh_longt         2480 non-null    float64       
 12  typh_lat           2480 non-null    float64       
 13  typh_no            2480 non-null    float64 

In [85]:
# 设置需要转换的列名列表
columns_to_convert = ['rnd', 'beta', 'clsprc', 'dps', 'lev', 'eps', 'bm', 'asset', 'debt', 'equity', 'roa', 'tang_asset_ratio', 'current_ratio', 'profit_volability', 'cash_ratio']

raw_df[columns_to_convert] = raw_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

## Indicator calculation
## CEF calculation

### GAMMA
Long-term Earnings Growth Rate 
Earnings growth rate = {Nth EPS - 1st year EPS}/{N-1} * 100%

In [86]:
gamma_df = raw_df[['id', 'time', 'eps']]

# 排序
gamma_df = gamma_df.sort_values(by=['id', 'time'])

gamma_df

,id,time,eps
0,1,2000,0.260328
1,1,2001,0.206782
2,1,2002,0.222130
3,1,2003,0.162680
4,1,2004,0.148921
...,...,...,...
125153,900957,2017,0.086005
125154,900957,2018,0.079327
125155,900957,2019,0.072611
125156,900957,2020,0.062632


In [87]:
# 定义一个计算gamma的函数
def calculate_gamma(x):
    if len(x) >1:
        first_eps = x.iloc[0]
        last_eps = x.iloc[-1]
        N = len(x)

        return (last_eps / first_eps) / (N-1)
    else:
        return None

gamma_df['gamma'] = gamma_df.groupby('id')['eps'].transform(calculate_gamma)

gamma_df

,id,time,eps,gamma
0,1,2000,0.260328,0.342502
1,1,2001,0.206782,0.342502
2,1,2002,0.222130,0.342502
3,1,2003,0.162680,0.342502
4,1,2004,0.148921,0.342502
...,...,...,...,...
125153,900957,2017,0.086005,0.017639
125154,900957,2018,0.079327,0.017639
125155,900957,2019,0.072611,0.017639
125156,900957,2020,0.062632,0.017639


In [88]:
check_dupl(gamma_df)

Empty DataFrame
Columns: [id, time, eps, gamma]
Index: []


In [89]:
gamma_df = gamma_df[['id', 'time', 'gamma']]

merged_df = pd.merge(raw_df,gamma_df, on=['id', 'time'], how='left')

# merged_df.to_csv('./merged/gamma_merged.csv', index=False)

# merged_df

In [90]:
raw_df = merged_df

### DELTA
delta represents the average dividend payout ratio of a stock over the last three years
Dividend payout ratio = dps/eps

In [91]:
delta_path = './DPS-EPS_yr_long.xlsx'

delta_df = pd.read_excel(delta_path, engine='openpyxl')

# delta_df

/Users/charles/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [92]:
delta_df = delta_df.drop([0,1])

delta_df['id'] = delta_df['Stkcd'].astype('int')
delta_df['time'] = pd.to_datetime(delta_df['Accper']).dt.year

delta_df = delta_df.rename(columns={'T20497': 'dps', 'T60200': 'eps'})

# delta_df


In [93]:
delta_df['dpr'] = np.where(delta_df['eps'] != 0, delta_df['dps']/delta_df['eps'], np.nan)

# delta_df

In [94]:
# 生成delta
delta_df['time_num'] = delta_df['time'].astype(int)

#进行排序
delta_df = delta_df.sort_values(by=['id', 'time'])

#计算过去3年的dpr均值
delta_df['delta'] = delta_df.groupby('id')['dpr'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# delta_df

In [95]:
check_dupl(delta_df)

Empty DataFrame
Columns: [Stkcd, Accper, dps, eps, id, time, dpr, time_num, delta]
Index: []


In [96]:
delta_df = delta_df[['id', 'time', 'delta']]

merged_df = pd.merge(raw_df, delta_df, on=['id', 'time'], how='left')

# merged_df

In [97]:
raw_df = merged_df

# raw_df

### EPS_1 EPS_2
EPS_1 is the analyst forecasted EPS for year t+1.

eps_2 represents analysts' forecasted EPS for year t+2

In [98]:
# 从三份文件中读取EPSPRD

# epsprd_dfs = {}
# for i in [1,2,3]:
#     epsprd_path = './EPSprd_' + str(i) + '.xlsx'
    
#     epsprd_dfs[f'epsprd_df{i}'] = pd.read_excel(epsprd_path, engine='openpyxl').drop([0,1])

In [99]:
# epsprd_df = pd.concat([epsprd_dfs['epsprd_df1'], epsprd_dfs['epsprd_df2'], epsprd_dfs['epsprd_df3']])

# epsprd_df

# epsprd_df.to_csv('./merged/EPSprd.csv', index=False)

In [100]:
# 读取数据并对EPSPRD数据进行处理
epsprd_df = pd.read_csv('./merged/EPSprd.csv')

epsprd_df['time'] = pd.to_datetime(epsprd_df['Rptdt']).dt.year
epsprd_df['prdyr'] = pd.to_datetime(epsprd_df['Fenddt']).dt.year

epsprd_df = epsprd_df.rename(columns={'Feps': 'epsprd', 'Stkcd': 'id'})

epsprd_df = epsprd_df[['id', 'time', 'prdyr', 'epsprd']]

# epsprd_df

In [101]:
# 取每个id-time-prdyr的epsprd均值
epsprd_df = epsprd_df.groupby(['id', 'time', 'prdyr'])['epsprd'].mean().reset_index(name= 'epsprd_ave')

# epsprd_df

In [102]:
# 读取表头
epsprd_hd_df = raw_df[['id', 'year_start', 'time']]

# epsprd_hd_df


In [103]:
eps_1_df = epsprd_hd_df

#生成一列“nextyear”
eps_1_df['next_yr'] = eps_1_df['year_start'] + pd.DateOffset(years=1)

eps_1_df['next_yr'] = eps_1_df['next_yr'].dt.year

# eps_1_df

/var/folders/20/dhknrks5505f4hwlc2p9vg500000gn/T/ipykernel_7789/3044587872.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps_1_df['next_yr'] = eps_1_df['year_start'] + pd.DateOffset(years=1)
/var/folders/20/dhknrks5505f4hwlc2p9vg500000gn/T/ipykernel_7789/3044587872.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps_1_df['next_yr'] = eps_1_df['next_yr'].dt.year


In [104]:
eps_1_merged_df = pd.merge(eps_1_df, epsprd_df, left_on=['id', 'time', 'next_yr'], right_on=['id', 'time', 'prdyr'], how='left')

# eps_1_merged_df.to_csv('./merged/eps_1_merged.csv', index=False)

eps_1_merged_df = eps_1_merged_df.rename(columns={'epsprd_ave': 'eps_1'})

eps_1_merged_df = eps_1_merged_df[['id', 'time', 'eps_1']]

# eps_1_merged_df

In [105]:
eps_1_merged_df = pd.merge(raw_df, eps_1_merged_df, on=['id', 'time'], how='left')

# eps_1_merged_df

In [106]:
check_dupl(eps_1_merged_df)

Empty DataFrame
Columns: [id, year_start, year_end, time, typh, obj_id, typh_date, typh_id, typh_name, typh_strg, typh_spd, typh_longt, typh_lat, typh_no, rnd, beta, clsprc, dps, lev, eps, bm, asset, debt, equity, roa, tang_asset_ratio, current_ratio, profit_volability, cash_ratio, industry, industry_id, soe, gamma, delta, eps_1]
Index: []

[0 rows x 35 columns]


In [107]:
raw_df = eps_1_merged_df

In [108]:
# 第2个年份的预测
eps_2_df = epsprd_hd_df

#生成一列“next2year”
eps_2_df['next2_yr'] = eps_2_df['year_start'] + pd.DateOffset(years=2)

eps_2_df['next2_yr'] = eps_2_df['next2_yr'].dt.year

# eps_2_df

/var/folders/20/dhknrks5505f4hwlc2p9vg500000gn/T/ipykernel_7789/3989004486.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps_2_df['next2_yr'] = eps_2_df['year_start'] + pd.DateOffset(years=2)
/var/folders/20/dhknrks5505f4hwlc2p9vg500000gn/T/ipykernel_7789/3989004486.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps_2_df['next2_yr'] = eps_2_df['next2_yr'].dt.year


In [109]:
eps_2_merged_df = pd.merge(eps_2_df, epsprd_df, left_on=['id', 'time', 'next2_yr'], right_on=['id', 'time', 'prdyr'], how='left')

# eps_2_merged_df.to_csv('./merged/eps_2_merged.csv', index=False)

eps_2_merged_df = eps_2_merged_df.rename(columns={'epsprd_ave': 'eps_2'})

eps_2_merged_df = eps_2_merged_df[['id', 'time', 'eps_2']]

# eps_2_merged_df

In [110]:
check_dupl(eps_2_merged_df)

Empty DataFrame
Columns: [id, time, eps_2]
Index: []


In [111]:
eps_2_merged_df = pd.merge(raw_df, eps_2_merged_df, on=['id', 'time'], how='left')

# eps_2_merged_df

In [112]:
raw_df = eps_2_merged_df

# raw_df

#### Completion of missing values
First assumption: if missing, then assume that eps_1 and eps are consistent; eps_2 and eps_1 are consistent

In [113]:
raw_df['eps_1'] = raw_df['eps_1'].fillna(raw_df['eps'])

# raw_df

In [114]:
raw_df['eps_2'] = raw_df['eps_2'].fillna(raw_df['eps_1'])

In [115]:
raw_df

,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,...,current_ratio,profit_volability,cash_ratio,industry,industry_id,soe,gamma,delta,eps_1,eps_2
0,1,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,金融、保险业,I,0.0,0.342502,0.559031,0.260328,0.260328
1,1,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,金融、保险业,I,0.0,0.342502,0.800832,0.206782,0.206782
2,1,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,金融、保险业,I,0.0,0.342502,0.466894,0.222130,0.222130
3,1,2003-01-01,2003-12-31,2003,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,金融、保险业,I,0.0,0.342502,0.466894,0.162680,0.162680
4,1,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,金融、保险业,I,0.0,0.342502,0.225093,0.244750,0.230000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.017639,0.000000,0.086005,0.086005
125154,900957,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.017639,0.000000,0.079327,0.079327
125155,900957,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.017639,0.000000,0.072611,0.072611
125156,900957,2020-01-01,2020-12-31,2020,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.017639,0.000000,0.062632,0.062632


### Calculate CEF according to the formula

In [116]:
print(raw_df.columns)

Index(['id', 'year_start', 'year_end', 'time', 'typh', 'obj_id', 'typh_date',
       'typh_id', 'typh_name', 'typh_strg', 'typh_spd', 'typh_longt',
       'typh_lat', 'typh_no', 'rnd', 'beta', 'clsprc', 'dps', 'lev', 'eps',
       'bm', 'asset', 'debt', 'equity', 'roa', 'tang_asset_ratio',
       'current_ratio', 'profit_volability', 'cash_ratio', 'industry',
       'industry_id', 'soe', 'gamma', 'delta', 'eps_1', 'eps_2'],
      dtype='object')


In [117]:
## 生成CEF
raw_df['cef_1'] = 0.5*((raw_df['gamma']-1)+(raw_df['delta']*raw_df['eps_1'])/raw_df['clsprc']) #第一项

raw_df['cef_2'] = (raw_df['cef_1'])**2 #第二项

raw_df['cef_3'] =  (raw_df['eps_1']/raw_df['clsprc'])*(((raw_df['eps_2'] - raw_df['eps_1'])/raw_df['eps_1'])- (raw_df['gamma'] - 1)) #第三项

raw_df['cef'] = raw_df['cef_1'] + (raw_df['cef_2']+raw_df['cef_3'])**0.5

#drop掉过程的几列
raw_df.drop(['cef_1','cef_2','cef_3'], axis=1, inplace= True)

raw_df

,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,...,profit_volability,cash_ratio,industry,industry_id,soe,gamma,delta,eps_1,eps_2,cef
0,1,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,金融、保险业,I,0.0,0.342502,0.559031,0.260328,0.260328,0.014774
1,1,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,金融、保险业,I,0.0,0.342502,0.800832,0.206782,0.206782,0.014181
2,1,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,金融、保险业,I,0.0,0.342502,0.466894,0.222130,0.222130,0.017877
3,1,2003-01-01,2003-12-31,2003,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,金融、保险业,I,0.0,0.342502,0.466894,0.162680,0.162680,0.015320
4,1,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,金融、保险业,I,0.0,0.342502,0.225093,0.244750,0.230000,0.025393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.017639,0.000000,0.086005,0.086005,0.071100
125154,900957,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.017639,0.000000,0.079327,0.079327,0.075987
125155,900957,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.017639,0.000000,0.072611,0.072611,0.106360
125156,900957,2020-01-01,2020-12-31,2020,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.017639,0.000000,0.062632,0.062632,0.102047


## lnSize Calculation

In [118]:
raw_df['asset'] = pd.to_numeric(raw_df['asset'], errors='coerce')

raw_df['ln_size'] = np.where(raw_df['asset'] > 0, np.log(raw_df['asset']), np.nan)

raw_df

/Users/charles/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,...,cash_ratio,industry,industry_id,soe,gamma,delta,eps_1,eps_2,cef,ln_size
0,1,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,金融、保险业,I,0.0,0.342502,0.559031,0.260328,0.260328,0.014774,24.931348
1,1,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,金融、保险业,I,0.0,0.342502,0.800832,0.206782,0.206782,0.014181,25.511815
2,1,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,金融、保险业,I,0.0,0.342502,0.466894,0.222130,0.222130,0.017877,25.836255
3,1,2003-01-01,2003-12-31,2003,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,金融、保险业,I,0.0,0.342502,0.466894,0.162680,0.162680,0.015320,25.985184
4,1,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,金融、保险业,I,0.0,0.342502,0.225093,0.244750,0.230000,0.025393,26.042789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125153,900957,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.017639,0.000000,0.086005,0.086005,0.071100,NaN
125154,900957,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.017639,0.000000,0.079327,0.079327,0.075987,NaN
125155,900957,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.017639,0.000000,0.072611,0.072611,0.106360,NaN
125156,900957,2020-01-01,2020-12-31,2020,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.017639,0.000000,0.062632,0.062632,0.102047,NaN


# Check data types

In [119]:
print(raw_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125158 entries, 0 to 125157
Data columns (total 38 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 125158 non-null  int64         
 1   year_start         125158 non-null  datetime64[ns]
 2   year_end           125158 non-null  datetime64[ns]
 3   time               125158 non-null  int64         
 4   typh               125158 non-null  float64       
 5   obj_id             2480 non-null    float64       
 6   typh_date          2480 non-null    float64       
 7   typh_id            2480 non-null    object        
 8   typh_name          2480 non-null    object        
 9   typh_strg          2480 non-null    float64       
 10  typh_spd           2480 non-null    float64       
 11  typh_longt         2480 non-null    float64       
 12  typh_lat           2480 non-null    float64       
 13  typh_no            2480 non-null    float64 

# Export summary table

In [133]:
raw_df.to_excel('./Yraw.xlsx', encoding='utf-8', index=False)

# Export short-form table

In [134]:
short_df = raw_df

short_df = short_df.groupby('id').filter(lambda x: (x['typh'] != 0).any())

short_df

,id,year_start,year_end,time,typh,obj_id,typh_date,typh_id,typh_name,typh_strg,...,industry,industry_id,soe,gamma,delta,eps_1,eps_2,cef,ln_size,lat
22,2,2000-01-01,2000-12-31,2000,0.0,NaN,NaN,NaN,NaN,NaN,...,房地产业,J,1.0,0.316194,0.313530,0.493171,0.493171,0.048291,22.449997,22.596869
23,2,2001-01-01,2001-12-31,2001,0.0,NaN,NaN,NaN,NaN,NaN,...,房地产业,J,1.0,0.316194,0.345381,0.605367,0.605367,0.041645,22.592436,22.596869
24,2,2002-01-01,2002-12-31,2002,0.0,NaN,NaN,NaN,NaN,NaN,...,房地产业,J,1.0,0.316194,0.337466,0.676667,0.803333,0.060693,22.829328,22.596869
25,2,2003-01-01,2003-12-31,2003,1.0,2368.0,2.003090e+09,2003Dujuan,Dujuan,4.0,...,房地产业,J,1.0,0.316194,0.256915,0.451000,0.563500,0.059631,23.080438,22.596869
26,2,2004-01-01,2004-12-31,2004,0.0,NaN,NaN,NaN,NaN,NaN,...,房地产业,J,1.0,0.316194,0.271350,0.473571,0.523333,0.077067,23.466324,22.596869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121259,835185,2017-01-01,2017-12-31,2017,0.0,NaN,NaN,NaN,NaN,NaN,...,采掘业,B,0.0,NaN,NaN,1.743400,1.915000,NaN,22.496112,22.800789
121260,835185,2018-01-01,2018-12-31,2018,0.0,NaN,NaN,NaN,NaN,NaN,...,采掘业,B,0.0,NaN,NaN,1.852000,2.219000,NaN,22.747131,22.800789
121261,835185,2019-01-01,2019-12-31,2019,0.0,NaN,NaN,NaN,NaN,NaN,...,采掘业,B,0.0,NaN,NaN,NaN,NaN,NaN,22.839860,22.800789
121262,835185,2020-01-01,2020-12-31,2020,1.0,2729.0,2.020082e+09,2020Higos,Higos,4.0,...,采掘业,B,0.0,NaN,0.292670,1.025044,1.025044,NaN,23.089373,22.800789


In [135]:
short_df.to_excel('./YrawSHORT.xlsx', encoding='utf-8', index=False)